## Download Images from Smithsonian API

In [1]:
#https://towardsdatascience.com/how-to-download-an-image-using-python-38a75cfa21c
import requests # to get image from the web
import shutil # to save it locally

In [24]:
import json
with open('data_raw.json') as json_file:
    data = json.load(json_file)

In [9]:
for d in data:
    filename = './images/' + d["title"] + '.jpg'
    image_url = d["image_link"]
    # Open the url image, set stream to True, this will return the stream content.
    r = requests.get(image_url, stream = True)
    
        # Check if the image was retrieved successfully
    if r.status_code == 200:
        # Set decode_content value to True, otherwise the downloaded image file's size will be zero.
        r.raw.decode_content = True

        # Open a local file with wb ( write binary ) permission.
        with open(filename,'wb') as f:
            shutil.copyfileobj(r.raw, f)

        print('Image sucessfully Downloaded: ',filename)
    else:
        print('Image Couldn\'t be retreived')

Image sucessfully Downloaded:  ./images/Bamboo Gate.jpg
Image sucessfully Downloaded:  ./images/Sausalito.jpg
Image sucessfully Downloaded:  ./images/The Blue Umbrella.jpg
Image sucessfully Downloaded:  ./images/Miss Plum Blossom.jpg
Image sucessfully Downloaded:  ./images/Head of Ah Ho.jpg
Image sucessfully Downloaded:  ./images/The Joggling Board.jpg
Image sucessfully Downloaded:  ./images/Little Miss Muffet.jpg
Image sucessfully Downloaded:  ./images/The Year at the Spring.jpg
Image sucessfully Downloaded:  ./images/A Child of the People.jpg
Image sucessfully Downloaded:  ./images/From the Rice Fields.jpg
Image sucessfully Downloaded:  ./images/On the Bund at Tokyo.jpg
Image sucessfully Downloaded:  ./images/A Roundelay.jpg
Image sucessfully Downloaded:  ./images/August.jpg
Image sucessfully Downloaded:  ./images/New Brooms.jpg
Image sucessfully Downloaded:  ./images/The Furious Dragon.jpg
Image sucessfully Downloaded:  ./images/A Japanese Madonna.jpg
Image sucessfully Downloaded:  

## Register Cropped Images in JSON

In [4]:
import glob
import re
import json

In [2]:
files = glob.glob("./images/cropping/*.jpg",recursive=True)
files

['./images/cropping/Plant62@Little Pink Plum.jpg',
 './images/cropping/Plant79@Moon Bridge at Kameido.jpg',
 "./images/cropping/Child48@The Goblins 'l Git Yer.jpg",
 './images/cropping/Child65@Mother and Child.jpg',
 './images/cropping/Figur74@A Monarch of Japan.jpg',
 './images/cropping/Child30@A Roundelay.jpg',
 './images/cropping/Tools34@August.jpg',
 './images/cropping/Figur23@From the Rice Fields.jpg',
 './images/cropping/Child85@The White Peacock.jpg',
 './images/cropping/Plant26@From the Rice Fields.jpg',
 './images/cropping/Child47@The Home of Gabriel.jpg',
 './images/cropping/Child63@Little Pink Plum.jpg',
 './images/cropping/Tools38@New Brooms.jpg',
 './images/cropping/Child29@A Roundelay.jpg',
 './images/cropping/Child1@Bamboo Gate.jpg',
 './images/cropping/Child6@Bamboo Gate.jpg',
 './images/cropping/Plant51@First Spring.jpg',
 './images/cropping/Plant57@Complaints.jpg',
 './images/cropping/Tools58@A Weary Little Mother.jpg',
 './images/cropping/Child33@August.jpg',
 './ima

In [ ]:
# >>> r = re.compile("([a-zA-Z]+)([0-9]+)")
# >>> m = r.match("foobar12345")
# >>> m.group(1)
# 'foobar'
# >>> m.group(2)
# '12345'

In [6]:
# Opening JSON file

data = []

with open('data.json') as json_file:
    data = json.load(json_file)


In [7]:
data_crop = []

r = re.compile("([a-zA-Z]+)([0-9]+)")

for f in files:
    m = r.match(f.split('/')[-1])
    title = f.split('/')[-1].split('@')[-1].split('.')[0]
    
    for d in data:
        if title == d["title"]:
            date = d["date"]
    
    item = {
        "itemid" : m.group(2),
        "group" : m.group(1),
        "title" : title,
        "date" : date,
        "root" : f
        
    }
    
    data_crop.append(item)


In [8]:
with open("data_crop.json", "w") as outfile:
    json.dump(data_crop, outfile, indent = 4)